# Cleaning and analysis of the given data

#### Imports

In [91]:
import re
import json
import csv

#### Variables

In [92]:
json_list = []
DATA_NAME = 'stat-2019-06.txt'

#### Functions

In [93]:
def check_bracket(json_string):
    counter, counter2, counter3 = 0, 0, 0
    counter += json_string.count("{")
    counter -= json_string.count("}")
    counter2 += json_string.count("[")
    counter2 -= json_string.count("]")
    counter3 += json_string.count("(")
    counter3 -= json_string.count(")")
    if counter == 0 and counter2==0 and counter3 == 0:
        return True
    print("SyntaxError: Bracket")
    return False

def check_syntax(json_string):
    try:
        json_str = json.loads(json_string)
        return True
    except:
        print("SyntaxError")
        return False

def check_elements(json_string):
    if json_string.find("\", \"sessionId\":\"")>=0:
        if json_string.find("\", \"robotName\":\"") >=0:
            if json_string.find("\", \"logLevel\":\"")>=0:
                if json_string.find("\", \"message\":")>=0:
                    return True
    print("Error: Element is missing or broken")
    return False

def validate_line(line):
    data_group = re.search("({.*})",line)
    try:
        json_string = data_group.group()
    except:
        return False, "o"
    if data_group and check_bracket(json_string) and check_syntax(json_string) and check_elements(json_string):
        json_dict = json.loads(data_group.group())
        return True, json_dict
    return False, "o"

## Reading of every usefull piece of data

In [94]:
with open(DATA_NAME,"rt") as data_file: 
    for line in data_file:
        if validate_line(line)[0]:
            json_list.append(validate_line(line)[1])


## Analysis 

#### Variables

In [95]:
list_actions = []
simulations = 0
robot_count = []
robot_names = []
stat_dict = {}

#### Create a dictionary sorted by robotName

In [96]:
for element in json_list:
    if type(element["message"]) is dict and element["message"]["action"]:
        if element["message"]["action"] not in list_actions:
            list_actions.append(element["message"]["action"])



for element in json_list:
    if element["robotName"] not in robot_names:
        robot_names.append(element["robotName"])
        stat_dict[element["robotName"]]={}
        for action in list_actions:
            stat_dict[element["robotName"]][action]=[]
        stat_dict[element["robotName"]]["other"]=[] 
    try:
        stat_dict[element["robotName"]][element["message"]["action"]].append(element)
    except:
        stat_dict[element["robotName"]]["other"].append(element)
                       



## Write CSV File

In [97]:
with open('stat_analysis_new.csv','w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([""]+robot_names)
    for action in list_actions:
        action_count_list = []
        action_count_list.append(action)
        for i in robot_names:
            action_count_list.append(len(stat_dict[i][action]))
        writer.writerow(action_count_list)

    